In [1]:
!git clone https://github.com/0xpantera/UFM-AI-22.git
%cd UFM-AI-22
from install import *
install_requirements()

Cloning into 'UFM-AI-22'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 80 (delta 34), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (80/80), done.
/content/UFM-AI-22
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


# Text Generation

Una de las características más extrañas de los modelos de lenguaje basados en transformers es su capacidad para generar texto que es casi indistinguible de texto escrito por humanos. Un ejemplo famoso es el de GPT-2 de OpenAI, que cuando se le presenta el prompt[1]:

> In a shocking finding, scientist discovered a herd of unicorns living in a remote, previ‐ ously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

genera un articulo que habla sobre unicornios:

> The scientist named the population, after their distinctive horn, Ovid’s Unicorn. These four-horned, silver-white unicorns were previously unknown to science. Now, after almost two centuries, the mystery of what sparked this odd phenomenon is finally solved. Dr. Jorge Pérez, an evolutionary biologist from the University of La Paz, and several companions, were exploring the Andes Mountains when they found a small valley, with no other animals or humans. Pérez noticed that the valley had what appeared to be a natural fountain, surrounded by two peaks of rock and silver snow. Pérez and the others then ventured further into the valley. “By the time we reached the top of one peak, the water looked blue, with some crystals on top,” said Pérez. Pérez and his friends were astonished to see the unicorn herd. These creatures could be seen from the air without having to move too much to see them—they were so close they could touch their horns. While examining these bizarre creatures the scientists discov‐ ered that the creatures also spoke some fairly regular English ...

Lo que hace que este ejemplo sea tan notable es que se generó sin ninguna supervisión explícita. Simplemente aprendiendo a predecir la siguiente palabra en el texto de millones de páginas web, GPT-2 y sus descendientes más poderosos como GPT-3 pueden aprender un gran conjunto de habilidades y capacidades de reconocimiento de patrones que se pueden activar con diferentes tipos de prompts. Los modelos de lenguaje a veces se exponen durante el pretraining a secuencias de tareas en las que necesitan predecir los siguientes tokens basándose únicamente en el contexto, como sumas, ordenar palabras desordenadas y traducir. Esto les permite hacer knowledge transfer de manera efectiva durante el finetunning o (si el modelo es lo suficientemente grande) a la hora de hacer inferencia. Estas tareas no se eligen con anticipación, sino que ocurren naturalmente en los enormes corpus utilizados para entrenar modelos de lenguaje de miles de millones de parámetros.

La capacidad de los transformers para generar texto realista ha dado lugar a una amplia gama de aplicaciones, como [InferKit](https://app.inferkit.com/demo), [Write With Transformer](https://transformer.huggingface.co/), [AI Dungeon](https://play.aidungeon.io/) y agentes conversacionales como [Meena de Google](https://ai.googleblog.com/2020/01/towards-conversational-agent-that-can.html) que incluso pueden contar chistes cursis[2].

En este notebook vamos a usar GPT-2 para ilustrar como la generacion de texto funciona para modelos de lenguajes y explorar como diferentes estrategias de decoding impactan el texto generado.

[1] Este ejemplo viene del [blog de OpenAI sobre GPT-2](https://openai.com/blog/better-language-models)

[2] Sin embargo, [como indica Delip Rao](https://twitter.com/deliprao/status/1479555633242259456), si Meena _pretende_ contar chistes cursis es una pregunta sutil.



Hasta ahora, en esta clase, nos hemos concentrado en abordar las tareas de NLP a través de una combinación de pretraining y finetunning supervisado. Como hemos visto, para las cabezas específicas a tareas como la clasificación de secuencias o tokens, generar predicciones es bastante sencillo; el modelo produce algunos logits y agarramos el valor máximo para obtener la clase predecida, o aplicamos una función softmax para obtener las probabilidades predecidas por clase. Por otro lado, convertir el output probabilístico del modelo en texto requiere un método de decoding, que tiene algunos desafíos que son exclusivos de la generación de texto:

- La decodificación se realiza de forma iterativa y, por lo tanto, implica mucho más cálculo que simplemente pasar los inputs una vez a través del forward pass de un modelo.
- La calidad y diversidad del texto generado depende de la elección del método de decodificación y los hiperparámetros asociados.

Para comprender cómo funciona este proceso de decodificación, comencemos viendoo cómo se preentrena GPT-2 y cómo se aplica posteriormente para generar texto.

Como otros modelos de lenguaje autoregresivos o causales, GPT-2 esta preentrenado para estimar la probabilidad $P(y|x)$ de una secuencia de tokens $y = y_1, y_2, ..., y_t$ que ocurren en un texto, dado un prompt inicial o una secuencia de contexto $x = x_1, x_2, ..., x_k$. Ya que es impractico adquirir suficiente data de entrenamiento para estimar $P(y|x)$ directamente, es comun usar la regla de la cadena de probabilidad para factorizarlo como un producto de probabilidades condicionales:

$P(y_1,...,y_t|x) = \prod^{N}_{t=1} P(y_t|y_{<t}, x)$

donde $y_{<t}$ es una notacion corta para la secuencia $y_1,...,y_{t-1}$. Es de estas probabilidades condicionales que agarramos la intuicion que el modelado de lenguaje autoregresivo equivale a predecir cada palabra dada la palabra que la precede en una oracion. Esto es exactamente lo que la probabilidad en el lado derecho de la equacion describe. Noten que este objetivo de preentrenamiento es diferente al de BERT, que usa contexto del pasado y futuro para predecir un token enmascarado.

Probablemente ya se dieron cuenta cómo podemos adaptar esta tarea de predicción de tokens para generar secuencias de texto de longitud arbitraria. Comenzamos con un mensaje como "Los transformers son los" y usamos el modelo para predecir el siguiente token. Una vez que hemos determinado el siguiente token, lo agregamos al prompt y luego usamos la nueva secuencia de entrada para generar otro token. Hacemos esto hasta que hayamos alcanzado un token especial de fin de secuencia o una longitud máxima predefinida.

> Dado que el output de una secuencia esta condicionado sobre la eleccion de input prompt, este tipo de generacion de texto se conoce como _conditional text generation_

En el corazón de este proceso hay un método de decodificación que determina qué token se selecciona en cada timestep. Dado que el modelo de lenguaje head produce un logit $z_{t,i}$ por token en el vocabulario en cada paso, podemos obtener la distribución de probabilidad sobre el próximo token $w_i$ tomando el softmax:

$P(y_t = w_i | y_{<t}, x) = softmax(z_{t,i})$

La meta de la mayoria de metodos de decodificacion es buscar la secuencia general mas probable eligiendo un $ŷ$ de modo que:

$ŷ = argmax P(y|x)$

Encontrar $ŷ$ directamente involucra evaluar cada posible secuencia con el modelo de lenguaje. Como no existe un algoritmo que pueda hacer esto en un tiempo razonable, nos apoyamos con aproximaciones. Vamos a explorar un par de estar aproximaciones y gradualmente llegar a un algoritmo mas inteligente y complejo que podamos usar para generar texto de alta calidad.

## Greedy Search Decoding

El método de decodificación más simple para obtener tokens discretos del output continu de un modelo es seleccionar _greedily_ (con avaricia? 🤷) el token con la mayor probabilidad en cada timestep:

$ŷ_t = argmax P(y_t|y_{<t}, x)$

Para ver como funciona greedy search, vamos a empezar cargando la version de GPT-2 con 1.5-billones de parametros con una cabeza de modelado de lenguaje:

> Si se quedan sin memoria, pueden probar cargar una version mas pequenia, reemplazando `model_name = "gpt-xl"` por `"gpt"`.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Ahora generemos un poco de texto. Aunque 🤗 Transformers provee una function `generate()` para modelos autoregresivos como GPT-2, vamos a implementar este metodo de decoding nosotros para ver como funciona. 

Para calentar, vamos a tomar un enfoque iterativo: usar "Transformers are the" como input prompt y ejecutar la decodificación durante ocho timesteps. En cada timestep, seleccionamos los logits del modelo para el último token en el prompt y los ajustamos con un softmax para obtener una distribución de probabilidad. Luego, elegimos el siguiente token con la probabilidad más alta, lo agregamos a la secuencia de entrada y ejecutamos el proceso nuevamente. El siguiente código hace el trabajo y también almacena los cinco tokens más probables en cada paso de tiempo para que podamos visualizar las alternativas:

In [2]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax 
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True) 
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (9.76%),same (2.94%),only (2.87%),best (2.38%),first (1.77%)
1,Transformers are the most,common (22.90%),powerful (6.88%),important (6.32%),popular (3.95%),commonly (2.14%)
2,Transformers are the most common,type (15.06%),types (3.31%),form (1.91%),way (1.89%),and (1.49%)
3,Transformers are the most common type,of (83.13%),in (3.16%),. (1.92%),", (1.63%)",for (0.88%)
4,Transformers are the most common type of,particle (1.55%),object (1.02%),light (0.71%),energy (0.67%),objects (0.66%)
5,Transformers are the most common type of particle,. (14.26%),in (11.57%),that (10.19%),", (9.57%)",accelerator (5.81%)
6,Transformers are the most common type of parti...,They (17.48%),\n (15.19%),The (7.06%),These (3.09%),In (3.07%)
7,Transformers are the most common type of parti...,are (38.78%),have (8.14%),can (7.99%),'re (5.04%),consist (1.57%)


Con este método simple pudimos generar la frase “Transformers are the most popular toy line in the world” (este fue el output de "gpt2-xl" pero este modelo hace que colab crashee. El "gpt" pequenio nos da un output mas malo). Curiosamente, esto indica que GPT-2 ha internalizado algunos conocimientos sobre la franquicia de Transformers, que fue creada por dos empresas de juguetes (Hasbro y Takara Tomy). También podemos ver las otras posibles continuaciones en cada paso, lo que muestra la naturaleza iterativa de la generación de texto. A diferencia de otras tareas, como la clasificación de secuencias, donde es suficiente con un solo forward pass para generar las predicciones, con la generación de texto necesitamos decodificar los tokens de salida uno a la vez.

Implementar greedy search no fue demasiado difícil, pero queremos usar la función `generar()` integrada de Transformers para explorar métodos de decodificación más sofisticados. Para reproducir nuestro ejemplo simple, asegurémonos de que el sampling esté desactivado (está desactivado de forma predeterminada, a menos que la configuración específica del modelo desde el que está cargando el checkpoint diga lo contrario) y especifiquemos `max_new_tokens` para la cantidad de tokens recién generados:

In [3]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device) 
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False) 
print(tokenizer.decode(output[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most common type of particle. They are


Ahora provemos algo mas interesante: podemos reproducir la historia de unicornios de OpenAI? Vamos a especificar un `max_length` mas grande para generar secuencias mas largas de texto:

In [4]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device) 
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False) 
print(tokenizer.decode(output_greedy[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very


Copio el output de "gp2-xl" que es mejor:

> In a shocking finding, scientist discovered a herd of unicorns living in a
    remote, previously unexplored valley, in the Andes Mountains. Even more
    surprising to the researchers was the fact that the unicorns spoke perfect
    English.
The researchers, from the University of California, Davis, and the University of
    Colorado, Boulder, were conducting a study on the Andean cloud forest, which is
    home to the rare species of cloud forest trees.
    The researchers were surprised to find that the unicorns were able to
    communicate with each other, and even with humans.
    The researchers were surprised to find that the unicorns were able


Bueno, las primeras oraciones son bastante diferentes del ejemplo de OpenAI e involucran a diferentes universidades a las que se les atribuye el descubrimiento. También podemos ver uno de los principales inconvenientes de la decodificación de greedy search: tiende a producir secuencias de output repetitivas, lo que fijo no es deseable en un artículo de noticias. Este es un problema común con los algoritmos de búsqueda _greedy_, que pueden fallar en darnos la solución óptima; en el contexto de la decodificación, pueden pasar por alto secuencias de palabras cuya probabilidad general es mayor simplemente porque las palabras de alta probabilidad están precedidas por otras de baja probabilidad.
Afortunadamente, podemos hacerlo mejor: examinemos un método popular conocido como _beam search decoding_.

> Aunque la decodificación de greedy search rara vez se usa para tareas de generación de texto que requieren diversidad, puede ser útil para producir secuencias cortas como aritmética donde se prefiere un output deterministico y objetivamente correcto.[3] Para estas tareas, pueden condicionar GPT-2 proporcionando algunos ejemplos separados por líneas en el formato "5 + 8 => 13 \n 7 + 2 => 9 \n 1 + 0 =>" como input prompt.

[3] N.S. Keskar et al., [“CTRL: A Conditional Transformer Language Model for Controllable Generation”](https://arxiv.org/abs/1909.05858), (2019).

## Beam Search Decoding

En lugar de decodificar el token con la probabilidad más alta en cada paso, beam search lleva un registro de los top-_b_ tokens mas probables, donde _b_ se conoce como el número de _beams_ o hipótesis parciales. El siguiente conjunto de beams se elige considerando todas las posibles extensiones del siguiente token del conjunto existente y seleccionando las b extensiones más probables. El proceso se repite hasta que alcanzamos la longitud máxima o un token EOS, y la secuencia más probable se selecciona clasificando los beams b según sus probabilidades logarítmicas.

Porque calificamos las secuencias usando probabilidades logartimicas en vez de probabilidades? Una razon es que calcular la probabilidad de una secuencia $P(y_1,...,y_t|x)$ involucra calcular un producto de probabilidades condicionales $P(y_t|y_{<t},x)$. Como cada probabilidad condicional es normalmente un numero pequenio en el rango [0,1], tomar sus productos puede llevarnos a una probabilidad que cause un underflow. Esto significa que la computadora ya no puede representar precisamente el resultado de los calculos. Por ejemplo, supongamos que tenemos una secuencia de $t = 1024$ tokens y asumamos que la probabilidad de cada token es 0.5. La probabilidad de la secuencia es un numero extremadamente pequenio:

In [5]:
0.5 ** 1024

5.562684646268003e-309

lo cual nos lleva a inestabilidad numerica cuando llegamos al underflow. Podemos evitar esto calculando un termino relacionado, la probabilidad logartimica. Si aplicamos el logaritmos a la probabilidad conjunta y condicional, con la ayuda de la regla de productos para logaritmos obtenemos:

$log P(y_1,...,y_t|x) = \sum^N_{t=1} log P(y_t|y_{<t}, x)$

En otras palabras, el producto de las probabilidades que vimos antes se vuelve una suma de probabilidades logartimicas, lo cual nos evita inestabilidad numerica. Por ejemplo, calcular la probabilidad logaritmica del mismo ejemplo nos da:

In [7]:
import numpy as np

sum([np.log(0.5)] * 1024)

-709.7827128933695

Este es un número con el que podemos lidiar fácilmente, y este enfoque aún funciona para números mucho más pequeños. Como solo queremos comparar probabilidades relativas, podemos hacerlo directamente con probabilidades logarítmicas.

Calculemos y comparemos las probabilidades logarítmicas de los textos generados por greedy search y por beam search para ver si beam search puede mejorar la probabilidad general. Dado que los modelos de 🤗 Transformers devuelven los logits no normalizados para el siguiente token dados los tokens de entrada, primero debemos normalizar los logits para crear una distribución de probabilidad sobre todo el vocabulario para cada token en la secuencia. Luego, debemos seleccionar solo las probabilidades de tokens que estaban presentes en la secuencia. La siguiente función implementa estos pasos:

In [8]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1) 
    return logp_label

Esto nos da la probabilidad logarítmica de un solo token, por lo que para obtener la probabilidad logarítmica total de una secuencia solo necesitamos sumar las probabilidades logarítmicas de cada token:

In [9]:
def sequence_logprob(model, labels, input_len=0): 
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

Tengan en cuenta que ignoramos las probabilidades logarítmicas de la secuencia de input porque el modelo no las genera. También podemos ver que es importante alinear los logits y las etiquetas; dado que el modelo predice el siguiente token, no obtenemos un logit para la primera etiqueta y no necesitamos el último logit porque no tenemos un token de verdad (ground truth) para él.

Usemos estas funciones para calcular primero la probabilidad logaritmica de la secuencia del greedy search decoder en el prompt de OpenAI:

In [10]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0])) 
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"The unicorns were very intelligent, and they were very intelligent," said Dr. David S. Siegel, a professor of anthropology at the University of California, Berkeley. "They were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very intelligent, and they were very

log-prob: -83.32


Ahora comparemos esto con una secuencia que se genera con beam search. Para activar beam search con la función `generate()` solo necesitamos especificar el número de beams con el parámetro `num_beams`. Cuantos más beams elijamos, mejor será potencialmente el resultado; sin embargo, el proceso de generación se vuelve mucho más lento ya que generamos secuencias paralelas para cada beam:

In [11]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0])) 
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the University of California, Santa Cruz, found that the unicorns were able to communicate with each other in a way that was similar to that of human speech.


"The unicorns were able to communicate with each other in a way that was similar to that of human speech," said study co-lead author Dr. David J.

log-prob: -78.34


Podemos ver que obtenemos una mejor probabilidad logaritmica (más alta es mejor) con beam search que con greedy decoding. Sin embargo, podemos ver que beam search también sufre de texto repetitivo. Una forma de abordar esto es imponer una penalización de n-gram con el parámetro `no_repeat_ngram_size` que rastrea qué n-grams se han visto y pone la probabilidad del siguiente token en cero si produce un n-gram visto anteriormente:

In [12]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0])) 
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, San Diego, and the National Science Foundation (NSF) in Boulder, Colorado, were able to translate the words of the unicorn into English, which they then translated into Spanish.

"This is the first time that we have translated a language into an English language," said study co-author and NSF professor of linguistics and evolutionary biology Dr.

log-prob: -101.87


Hemos logrado detener las repeticiones y podemos ver que a pesar de producir una puntuación más baja, el texto sigue siendo coherente. Beam search con penalización de n-grams es una buena manera de encontrar un equilibrio entre centrarse en tokens de alta probabilidad (con beam search) y reducir las repeticiones (con penalización de n-grams), y se usa comúnmente en aplicaciones como resúmenes o traducción automática donde la exactitud de los hechos es importante. Cuando la exactitud de los hechos es menos importante que la diversidad de los resultados generados, por ejemplo, en una conversacion de dominio abierto o en la generación de historias, otra alternativa para reducir las repeticiones y mejorar la diversidad es usar el muestreo.

## Metodos de Muestro

TBC